In [1]:
print('1')

1


In [7]:
!curl "http://prometheus-grafana.default.svc.cluster.local:80"


curl: (6) Could not resolve host: prometheus-grafana.default.svc.cluster.local


In [14]:
import requests
from datetime import datetime, timedelta

def get_pod_metrics(pod_name, service_name):
    try:
        url = f"{service_name}/api/v1/query"
        _pod_name = "pod"
        cpu_usage_query_string, mem_usage_querystring = create_prometheus_payload(pod_name, {})


        print('cpu url - ', url)
        print('params - ', cpu_usage_query_string)
        mem_response = requests.get(url, params=mem_usage_querystring)
        cpu_response = requests.get(url, params=cpu_usage_query_string)
        print('p mem cpu')
        print(mem_response.json())
        print(cpu_response.json())

        mem_response.raise_for_status()
        cpu_response.raise_for_status()
        if (
                cpu_response.json()["data"]["result"] and mem_response.json()["data"]["result"]
        ):
            pod_metrics = {
                "pod": cpu_response.json()["data"]["result"][0]["metric"][f"{_pod_name}"],
                "cpu_utilization": {
                    "time": cpu_response.json()["data"]["result"][0]["value"][0],
                    "cpu_percent": cpu_response.json()["data"]["result"][0]["value"][1],
                },
                "mempry_utilization": {
                    "time": mem_response.json()["data"]["result"][0]["value"][0],
                    "memory": mem_response.json()["data"]["result"][0]["value"][1],
                },
            }
        else:
            pod_metrics = {}
        return pod_metrics
    except Exception as ex:
        print(ex)
        raise ValueError("unable to fetch pod metrics")


def create_prometheus_payload(pod_name, additional_params):
    """
    Method returns common payload required for prometheus api
    :param pod_name: K8s pod name
    :param additional_params: optional keys - step, end, start
    :return:
    """
    _pod_name = "pod"
    _container_name = "container"

    # Specify the time range for the query
    end_time = datetime.now()  # current time
    start_time = end_time - timedelta(minutes=5)  # 5 minutes ago

    cpu_usage_query_string = {
        "query": "sum(rate(container_cpu_usage_seconds_total{"
                 f"{_container_name}!='POD',"
                 f"namespace!='',{_pod_name}="
                 f"'{pod_name}'"
                 f",{_container_name}!='', "
                 f"{_container_name}!="
                 "'knights-watch'}[2m]))"
                 f" by ({_pod_name})",
        "start": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "end": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "step": "1m"
    }
    mem_usage_querystring = {
        "query": "sum(container_memory_usage_bytes{"
                 f"{_container_name}!='POD',namespace!='',"
                 f"{_pod_name}="
                 f"'{pod_name}'"
                 f",{_container_name}!='', "
                 f"{_container_name}!="
                 "'knights-watch'})"
                 f" by ({_pod_name}) / 1000000",
        "start": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "end": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "step": "1m"
    }
    cpu_usage_query_string.update(additional_params)
    mem_usage_querystring.update(additional_params)
    return cpu_usage_query_string, mem_usage_querystring


service_name = "http://prometheus-grafana-kube-pr-prometheus.monitoring"

pod_name = "jy-d39533af-1ce1-4c11-93f7-a9469d18042a-fdcuserfosforcom-3e0e"

final_response = get_pod_metrics(pod_name, service_name)


print(final_response)

cpu url -  http://prometheus-grafana-kube-pr-prometheus.monitoring/api/v1/query
params -  {'query': "sum(rate(container_cpu_usage_seconds_total{container!='POD',namespace!='',pod='jy-d39533af-1ce1-4c11-93f7-a9469d18042a-fdcuserfosforcom-3e0e',container!='', container!='knights-watch'}[2m])) by (pod)", 'start': '2024-05-31T06:14:46Z', 'end': '2024-05-31T06:19:46Z', 'step': '1m'}
p mem cpu
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'pod': 'jy-d39533af-1ce1-4c11-93f7-a9469d18042a-fdcuserfosforcom-3e0e'}, 'value': [1717136386.926, '192.88064']}]}}
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'pod': 'jy-d39533af-1ce1-4c11-93f7-a9469d18042a-fdcuserfosforcom-3e0e'}, 'value': [1717136386.952, '0.00018303938154429875']}]}}
{'pod': 'jy-d39533af-1ce1-4c11-93f7-a9469d18042a-fdcuserfosforcom-3e0e', 'cpu_utilization': {'time': 1717136386.952, 'cpu_percent': '0.00018303938154429875'}, 'mempry_utilization': {'time': 1717136386.926, 'm